<h1><center> UFU - Federal University of Uberlândia</center></h1>

<h2><center>Undergraduate Program in Civil Engineering</center></h2>

<h3><center>SCIENTIFIC RESEARCH PROJECT</center><br>
TITLE: USING XGBOOST MODELS FOR DAILY RAINFALL PREDICTION 
<br>  
<br>  
STUDENT: Pedro Augusto Toledo Rios</h3>

<p>This notebook is part of a Scientific Research Project in the field of Computer Science/Data Analysis.</p>


# Classification

## Imports and Initial Configurations

In [ ]:
# Data Analysis and Wrangling
import pandas as pd
import numpy as np
import random as rnd
import seaborn as sn

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
import itertools
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import ConfusionMatrixDisplay
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt


# Exploratory Data Analysis

In [ ]:
# Data Preprocessing
missing_values = [
    'n/a', 'na', '*****', '*', '*******', ' -', '******', '5..84', '3..66', 
    '3.3.21', '1..41', '********', '3.7.94', '354.59*', '564..79', '5.04.24', 
    '21:36', '**********', '***', '*********', '03:18', '00:00', '03:48', 
    '08:42', '03:06', '09:06', '01:30', '07:48', '09:12', '10:18', '01:24', 
    '#VALUE!', '926,4923,8', '27/07/1902**21:36:00', '-', '*****', '185.488.992'
]

# Load dataset and define column names
climate_data = pd.read_csv(
    "C:/Users/auped/Desktop/IC CORREÇÕES/python 05-11/Titanic/modelo global/Classificação/dadosclima_PERIODOSECO - Copia.csv",
    header=None,
    sep=';',
    na_values=missing_values
)

climate_data.columns = [
    'Max Temp (°C)', 'Min Temp (°C)', 'Avg Temp (°C)', 'Wind Speed (km/h)', 
    'Solar Radiation (cal/cm²/h)', 'Pressure (mb)', 'Humidity (%)', 'Rainfall (mm)', 
    'Month', 'Year', 'Rainy/Dry'
]

# Data Cleaning
climate_data['Pressure (mb)'] = climate_data['Pressure (mb)'].str.replace(',,', '.')
climate_data['Humidity (%)'] = climate_data['Humidity (%)'].astype(float)
climate_data['Pressure (mb)'] = climate_data['Pressure (mb)'].astype(float)
climate_data['Year'] = climate_data['Year'].str.replace(',,', '')
climate_data['Year'] = climate_data['Year'].astype(int)

# Filtering pressure values within the valid range
climate_data = climate_data[(climate_data['Pressure (mb)'] >= 870) & (climate_data['Pressure (mb)'] <= 1100)]

# Storing rainfall data separately
combine = [climate_data]
rainfall = climate_data['Rainfall (mm)']

# Display first five rows of the processed dataset
climate_data.head()
climate_data.dropna()

# Display the number of missing values per column
print(climate_data.isnull().sum())


In [ ]:
climate_data.describe()

In [ ]:
# Creating a new column indicating whether it rained
# Using the existing 'Rainfall (mm)' column

for climate_data in combine:  # Changed from 'combine' to 'dados_clima' to match your dataset variable
    climate_data.loc[climate_data['Rainfall (mm)'] == 0, 'Rained?'] = 0
    climate_data.loc[climate_data['Rainfall (mm)'] > 0, 'Rained?'] = 1

# Display the first five rows of the dataset
    climate_data.head()

In [ ]:
# Handling Missing Values

# Forward-fill (ffill) method: fills missing values with the last available value in the column
climate_data['Min Temp (°C)'].fillna(method='ffill', inplace=True)
climate_data['Avg Temp (°C)'].fillna(method='ffill', inplace=True)
climate_data['Wind Speed (km/h)'].fillna(method='ffill', inplace=True)
climate_data['Solar Radiation (cal/cm²/h)'].fillna(method='ffill', inplace=True)
climate_data['Humidity (%)'].fillna(method='ffill', inplace=True)
climate_data['Rainfall (mm)'].fillna(method='ffill', inplace=True)
climate_data['Pressure (mb)'].fillna(method='ffill', inplace=True)
climate_data['Rained?'].fillna(method='ffill', inplace=True)
climate_data['Max Temp (°C)'].fillna(method='ffill', inplace=True)

# Checking for missing values after imputation
print('\nMissing Values in the dataset after treatment:\n', climate_data.isnull().sum(), sep="")


In [ ]:
climate_data.describe()

In [ ]:
# 70% for training / 30% for testing

start_year = 1980
end_year = 2020
training_start_year = 1980
training_end_year = 2008
testing_start_year = 2009
testing_end_year = 2019

# Creating a copy of the dataset for testing
test_data = climate_data.copy()


In [ ]:
# Removing the "Rain (mm)" column from the dataset  
test_data.drop(columns=['Rainfall (mm)'], axis=1, inplace=True)


# Machine Learning Models


## Creating Training and Testing DataFrames

In [ ]:
# Creating training and test datasets
df_train = test_data[test_data['Year'] >= training_start_year]
df_train = test_data[test_data['Year'] <= training_end_year]
df_test = test_data[test_data['Year'] >= testing_start_year]
df_test = test_data[test_data['Year'] <= testing_end_year]



# Splitting features (X) and target variable (Y)
x_train = df_train.iloc[:, 0:8]
y_train = df_train[['Rained?']]

x_test = df_test.iloc[:, 0:8]
y_test = df_test[['Rained?']]

# Displaying the first few rows of the test feature set
x_test.head()


## Model - XGBoost to Determine Whether It Rained or Not


In [ ]:
# Creating the XGBoost classifier object
xgboost_classifier = xgb.XGBClassifier()

# Training the XGBoost classifier
xgb_model = xgboost_classifier.fit(x_test, y_test)

# Making predictions on the training set
y_pred_train = xgb_model.predict(x_train)


In [ ]:
print('Precision: %.3f' % precision_score(y_train, y_pred_train))
print('Accuracy: %.3f' % accuracy_score(y_train, y_pred_train))
print('Recall: %.3f' % recall_score(y_train, y_pred_train))

In [ ]:
# Compute the confusion matrix
cm = confusion_matrix(y_train, y_pred_train)

# Define class labels (replace with your own if necessary)
class_names = ['Negative Class', 'Positive Class']

# Plot the confusion matrix using Seaborn
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2)  # Adjust font size

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)

plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()
